In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [22]:
sa2_df = pd.read_csv("resources/affordability.csv")
sa2_df.min()

Unnamed: 0                                                   0
SA2 Name                         Alexander Heights - Koondoola
median_income of suburb                                1189.17
median_rent                                            278.125
median_mortgage_repayment                              348.356
unemployment_rate                                    0.0466526
unoccupied_rate                                      0.0528926
yearly building_approval                                     1
Greater Perth median income                            2333.17
qualyfing_income(mortgage)                             1161.19
affordability_index(mortgage)                          77.5362
qualyfing_income(rent)                                 927.083
affordability_index(rent)                               90.395
dtype: object

In [19]:
gmaps.configure(api_key=g_key)

In [25]:
sa2_coordinates = {"SA2 Name":[],"lat":[],"lng":[]}
for address in sa2_df["SA2 Name"]:
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    parameter = {"address":address +" ,WA", "key":g_key}
    response = requests.get(url, params = parameter)
    data = response.json()
    try:
        sa2_coordinates["lat"].append(data["results"][0]["geometry"]["location"]["lat"])
        sa2_coordinates["lng"].append(data["results"][0]["geometry"]["location"]["lng"])    
        sa2_coordinates["SA2 Name"].append(address)
    except:
        print(f'{address} has error.')




In [26]:
coordinates_df = pd.DataFrame(sa2_coordinates)

In [27]:
add_df = pd.read_csv("resources/affordability.csv")

In [50]:
gmaps_df = pd.merge(coordinates_df, add_df, on="SA2 Name", how="inner")
gmaps_df.max()

SA2 Name                         Yokine - Coolbinia - Menora
lat                                                  48.9311
lng                                                  116.168
Unnamed: 0                                               147
median_income of suburb                              3754.78
median_rent                                          774.324
median_mortgage_repayment                             902.74
unemployment_rate                                   0.142461
unoccupied_rate                                     0.314557
yearly building_approval                                 877
Greater Perth median income                          2333.17
qualyfing_income(mortgage)                           3009.13
affordability_index(mortgage)                        200.929
qualyfing_income(rent)                               2581.08
affordability_index(rent)                            251.668
dtype: object

In [32]:

locations = gmaps_df[["lat", "lng"]].astype(float)
median_rent = gmaps_df["median_rent"]

In [51]:
# Add marker layer ontop of heat map

heat_layer = gmaps.heatmap_layer(locations, weights=median_rent, 
                                 dissipating=False, max_intensity=800,
                                 point_radius = 0.02)
# Display figure
fig = gmaps.figure(center=(-31.9,116),zoom_level=11)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))